We decided to work with 2 datasets: all info about athletes participated in Olympic Games and the second one is about the code of their regions (countries)

First, we import csv library to work with csv files:

In [1]:
import csv
with open('athlete_events.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    print(type(reader))
    print(reader.fieldnames) # to get column names

<class 'csv.DictReader'>
['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team', 'NOC', 'Games', 'Year', 'Season', 'City', 'Sport', 'Event', 'Medal']


In order to not open the file everytime we need the data, we store the data in list:

In [2]:
with(open('athlete_events.csv', 'r')) as csvfile:
    reader = csv.DictReader(csvfile)
    athletes = []
    for row in reader:
        athletes.append(row)
    print("Number athletes:", len(athletes)) # number of all participants

Number athletes: 271116


In [3]:
athletes[:1] # example of the data of athletes

[{'ID': '1',
  'Name': 'A Dijiang',
  'Sex': 'M',
  'Age': '24',
  'Height': '180',
  'Weight': '80',
  'Team': 'China',
  'NOC': 'CHN',
  'Games': '1992 Summer',
  'Year': '1992',
  'Season': 'Summer',
  'City': 'Barcelona',
  'Sport': 'Basketball',
  'Event': "Basketball Men's Basketball",
  'Medal': 'NA'}]

The main idea of the code below is to demonstrate the number of participants in 1976 year:

In [4]:
athletes_1976 = [row for row in athletes if row["Year"] == "1976"]
print("Number athletes in 1976:", len(athletes_1976))

Number athletes in 1976: 10502


In [5]:
# We used this function for our purposes (to filter the year column)
# Author Prof. Samer El Zant
def is_valid_year(string):
    try:
        year = int(string)
    except ValueError:
        return False
    else:
        return year

In [6]:
release_years = [int(row['Year']) for row in athletes if is_valid_year(row['Year'])]
print(min(release_years)) # since which year our records begin

1896


In [46]:
print(max(release_years)) # since which year our records begin

2016


We used Collection library for counting our data:

In [7]:
from collections import Counter, OrderedDict

athletes_year = Counter(el['Year'] for el in athletes) # for counting the years
athletes_year = dict(athletes_year) # changing type
athletes_year = OrderedDict(sorted(athletes_year.items())) # sorting
athletes_year

OrderedDict([('1896', 380),
             ('1900', 1936),
             ('1904', 1301),
             ('1906', 1733),
             ('1908', 3101),
             ('1912', 4040),
             ('1920', 4292),
             ('1924', 5693),
             ('1928', 5574),
             ('1932', 3321),
             ('1936', 7401),
             ('1948', 7480),
             ('1952', 9358),
             ('1956', 6434),
             ('1960', 9235),
             ('1964', 9480),
             ('1968', 10479),
             ('1972', 11959),
             ('1976', 10502),
             ('1980', 8937),
             ('1984', 11588),
             ('1988', 14676),
             ('1992', 16413),
             ('1994', 3160),
             ('1996', 13780),
             ('1998', 3605),
             ('2000', 13821),
             ('2002', 4109),
             ('2004', 13443),
             ('2006', 4382),
             ('2008', 13602),
             ('2010', 4402),
             ('2012', 12920),
             ('2014', 4891),
    

By this result, we can see some gaps in years: 1912-1920 were no games because there was a world war first during 1914-1918, and also in the period of 1936-1948, also because of war (WW2) during 1939-1945.

Now we are introducing our second dataset with regions. To mention, it has one common column with our first dataset, NOC column:

In [8]:
with(open('noc_regions.csv', 'r')) as csvfile:
    reader = csv.DictReader(csvfile)
    print(reader.fieldnames)
    regions = []
    for row in reader:
        regions.append(row)
    print("Number regions:", len(regions))

['NOC', 'region', 'notes']
Number regions: 230


Now we are joining the Regions dataset to our main dataset:

In [9]:
noc_dict = {reg['NOC']:reg for reg in regions}

merged = []
for ath in athletes:
    d = dict(**ath)
    d.update(noc_dict.get(ath['NOC'], {}))
    merged.append(d)

merged[0]

{'ID': '1',
 'Name': 'A Dijiang',
 'Sex': 'M',
 'Age': '24',
 'Height': '180',
 'Weight': '80',
 'Team': 'China',
 'NOC': 'CHN',
 'Games': '1992 Summer',
 'Year': '1992',
 'Season': 'Summer',
 'City': 'Barcelona',
 'Sport': 'Basketball',
 'Event': "Basketball Men's Basketball",
 'Medal': 'NA',
 'region': 'China',
 'notes': ''}

Now we are doing the data analysis.

For counting the total amount of medals:

In [10]:
medals = 0
for row in athletes:
  if row['Medal'] != "NA":
    medals+=1
print("The total of the medals: ", medals)

The total of the medals:  39783


Now amount of medals of each type:

In [18]:
# We reduced the code with Counter to one line.
OrderedDict(sorted(dict(Counter(t['Medal'] for t in merged)).items()))

OrderedDict([('Bronze', 13295),
             ('Gold', 13372),
             ('NA', 231333),
             ('Silver', 13116)])

In [48]:
OrderedDict(sorted(dict(Counter(t['Medal'] for t in merged if t['NOC']== 'AZE')).items()))

OrderedDict([('Bronze', 25), ('Gold', 7), ('NA', 239), ('Silver', 12)])

For counting the amount of athletes participated in each season.

In [12]:
OrderedDict(sorted(dict(Counter(t['Season'] for t in merged)).items()))

OrderedDict([('Summer', 222552), ('Winter', 48564)])

For counting how many male and female athletes participated in each season.

In [13]:
print(OrderedDict(sorted(dict(Counter(t['Sex'] for t in merged)).items())))

OrderedDict([('F', 74522), ('M', 196594)])


In [14]:
m_s = 0 # male in summer
m_w = 0 # male in winter
f_s = 0 # female in summer
f_w = 0 # female in winter
for row in athletes:
  if row["Sex"]=="M" and row["Season"] == "Summer":
    m_s+=1
  if row["Sex"]=="M" and row["Season"] == "Winter":
    m_w+=1
  if row["Sex"]=="F" and row["Season"] == "Summer":
    f_s+=1
  if row["Sex"]=="F" and row["Season"] == "Winter":
    f_w+=1

print("The number of males in summer game: ", m_s)
print("The number of males in winter game: ", m_w)
print("The number of females in summer game: ", f_s)
print("The number of females in winter game: ", f_w)

The number of males in summer game:  163109
The number of males in winter game:  33485
The number of females in summer game:  59443
The number of females in winter game:  15079


Amount of athletes of each region in descending order:

In [29]:
OrderedDict(sorted(dict(Counter(t['NOC'] for t in merged)).items(), key=lambda x: x[1], reverse=True))

OrderedDict([('USA', 18853),
             ('FRA', 12758),
             ('GBR', 12256),
             ('ITA', 10715),
             ('GER', 9830),
             ('CAN', 9733),
             ('JPN', 8444),
             ('SWE', 8339),
             ('AUS', 7638),
             ('HUN', 6607),
             ('POL', 6207),
             ('SUI', 6150),
             ('NED', 5839),
             ('URS', 5685),
             ('FIN', 5467),
             ('ESP', 5313),
             ('RUS', 5143),
             ('CHN', 5141),
             ('AUT', 5141),
             ('NOR', 4960),
             ('KOR', 4464),
             ('ROU', 4405),
             ('TCH', 4404),
             ('BEL', 3857),
             ('BRA', 3848),
             ('DEN', 3570),
             ('BUL', 3530),
             ('FRG', 3315),
             ('ARG', 3297),
             ('GRE', 3181),
             ('MEX', 2880),
             ('GDR', 2645),
             ('YUG', 2583),
             ('UKR', 2559),
             ('CUB', 2479),
             ('N

Amount of participants from Azerbaijan:

In [47]:
OrderedDict(sorted(dict(Counter(t['NOC'] for t in merged if t['NOC']== 'AZE')).items(), key=lambda x: x[1], reverse=True))

OrderedDict([('AZE', 283)])

The first year when Azerbaijan participated in OG:

In [49]:
release_years_az = [int(row['Year']) for row in athletes if is_valid_year(row['Year']) and row['NOC'] == "AZE"]
print(min(release_years_az)) # since which year our records begin

1996


Amount of medals of each type won by females:

In [22]:
OrderedDict(sorted(dict(Counter(t['Medal'] for t in merged if t['Sex'] == 'F')).items()))

OrderedDict([('Bronze', 3771),
             ('Gold', 3747),
             ('NA', 63269),
             ('Silver', 3735)])

Amount of medals of each type won by males:

In [23]:
OrderedDict(sorted(dict(Counter(t['Medal'] for t in merged if t['Sex'] == 'M')).items()))

OrderedDict([('Bronze', 9524),
             ('Gold', 9625),
             ('NA', 168064),
             ('Silver', 9381)])

Best three countries by amount of gold medals for each region:

In [44]:
list(OrderedDict(sorted(dict(Counter(t['NOC'] for t in merged if t['Medal'] == 'Gold')).items(),key=lambda x: x[1], reverse=True)).items())[0:3]

[('USA', 2638), ('URS', 1082), ('GER', 745)]

In [50]:
list(OrderedDict(sorted(dict(Counter(t['NOC'] for t in merged if t['Medal'] == 'Gold')).items(),key=lambda x: x[1], reverse=True)).items())

[('USA', 2638),
 ('URS', 1082),
 ('GER', 745),
 ('GBR', 678),
 ('ITA', 575),
 ('FRA', 501),
 ('SWE', 479),
 ('CAN', 463),
 ('HUN', 432),
 ('GDR', 397),
 ('RUS', 390),
 ('NOR', 378),
 ('CHN', 350),
 ('AUS', 348),
 ('NED', 287),
 ('JPN', 247),
 ('KOR', 221),
 ('FIN', 198),
 ('DEN', 179),
 ('SUI', 175),
 ('CUB', 164),
 ('ROU', 161),
 ('FRG', 159),
 ('IND', 138),
 ('YUG', 130),
 ('EUN', 127),
 ('POL', 117),
 ('ESP', 110),
 ('BRA', 109),
 ('AUT', 108),
 ('BEL', 98),
 ('ARG', 91),
 ('NZL', 90),
 ('TCH', 81),
 ('GRE', 62),
 ('CRO', 58),
 ('BUL', 54),
 ('UKR', 47),
 ('PAK', 42),
 ('CZE', 42),
 ('TUR', 40),
 ('JAM', 38),
 ('KEN', 34),
 ('RSA', 32),
 ('URU', 31),
 ('MEX', 30),
 ('BLR', 24),
 ('NGR', 23),
 ('ETH', 22),
 ('CMR', 20),
 ('KAZ', 20),
 ('ANZ', 20),
 ('IRI', 18),
 ('ZIM', 17),
 ('PRK', 16),
 ('SRB', 15),
 ('SVK', 15),
 ('BAH', 14),
 ('FIJ', 13),
 ('EST', 13),
 ('SCG', 12),
 ('INA', 11),
 ('UZB', 10),
 ('THA', 9),
 ('IRL', 9),
 ('GEO', 8),
 ('SLO', 8),
 ('AZE', 7),
 ('TTO', 7),
 ('EGY',

Best three countries by amount of silver medals for each region:

In [45]:
list(OrderedDict(sorted(dict(Counter(t['NOC'] for t in merged if t['Medal'] == 'Silver')).items(),key=lambda x: x[1], reverse=True)).items())[0:3]

[('USA', 1641), ('GBR', 739), ('URS', 732)]

Best three countries by amount of bronze medals for each region:

In [43]:
list(OrderedDict(sorted(dict(Counter(t['NOC'] for t in merged if t['Medal'] == 'Bronze')).items(),key=lambda x: x[1], reverse=True)).items())[0:3]

[('USA', 1358), ('GER', 746), ('URS', 689)]

Amount of medals obtained by each country:

In [30]:
OrderedDict(sorted(dict(Counter(t['NOC'] for t in merged if t['Medal'] != 'NA')).items(), key=lambda x: x[1], reverse=True))

OrderedDict([('USA', 5637),
             ('URS', 2503),
             ('GER', 2165),
             ('GBR', 2068),
             ('FRA', 1777),
             ('ITA', 1637),
             ('SWE', 1536),
             ('CAN', 1352),
             ('AUS', 1320),
             ('RUS', 1165),
             ('HUN', 1135),
             ('NED', 1040),
             ('NOR', 1033),
             ('GDR', 1005),
             ('CHN', 989),
             ('JPN', 913),
             ('FIN', 900),
             ('SUI', 691),
             ('ROU', 653),
             ('KOR', 638),
             ('DEN', 597),
             ('FRG', 586),
             ('POL', 565),
             ('ESP', 489),
             ('TCH', 488),
             ('BRA', 475),
             ('BEL', 468),
             ('AUT', 450),
             ('CUB', 409),
             ('YUG', 390),
             ('BUL', 342),
             ('EUN', 279),
             ('ARG', 274),
             ('GRE', 255),
             ('NZL', 228),
             ('UKR', 199),
             (